In [4]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [5]:
from time import gmtime, strftime

In [ ]:
import doctest
doctest.testmod()

In [6]:
from pyspark import StorageLevel
import pydoop.hdfs as hdfs

In [3]:
from math import log, exp
from functools import reduce

"""
sources: 
http://windowoffice.tumblr.com/post/33548509/logsum-underflow-trick-re-discovery
https://facwiki.cs.byu.edu/nlp/index.php/Log_Domain_Computations
https://mikelove.wordpress.com/2011/06/06/log-probabilities-trick/
"""
def log_add(logx,logy):

    if (logy > logx):
        logy,logx = logx,logy
        
    negdiff = logy - logx

    if negdiff < -20:
        return logx

    return logx + log(1.0 + exp(negdiff))

def sum_logs(logs):
    """
    return the log of total values
    >>> exp(sum_logs([log(0.1),log(0.4),log(0.6)]))
    1.1
    """
    return reduce(log_add,logs) 

def normalize_logs(logs):
    """
    convert the logs to normalized probablities
    >>>normalize_logs(array([-1000,-1000,-990]))
    array([  4.53958078e-05,   4.53958078e-05,   9.99909208e-01])
    """
    log_total = sum_logs(logs)
    logprobs = logs-log_total
    probs = np.exp(logprobs)
    return probs

def pick_random_points(logprobs,num_points):
    """
    picks random entries according to the logprobs vector
    >>>np.random.seed(24)
    >>>pick_random_points(array([log(0.1),log(0.1),log(0.5),log(0.3)]),num_points=10)
    array([3, 2, 3, 2, 2, 3, 3, 2, 1, 2])
    """
    probs = normalize_logs(logprobs)
    xk = np.arange(len(probs))
    custm = stats.rv_discrete(name='custm', values=(xk, probs))
    return custm.rvs(size=num_points)

def logprobs_to_normprobs(logprobs):
    probs = np.exp(logprobs)
    norm_prbs = probs/np.sum(probs)
    return norm_prbs

def logprobs_to_normprobs_safe(logprobs):
#     this version uses log addition
    logprob_total = sum_logs(logprobs)
    norm_logprbs = logprobs-logprob_total
    norm_prbs = np.exp(norm_logprbs)
    return norm_prbs

def calc_exp_log_prob(probs,logprobs):
    return np.sum([0 if prob<=0 else prob*logprob for prob,logprob in zip(probs,logprobs)])

In [1]:
from pyspark import SparkContext, SparkConf
sc = SparkContext(conf = SparkConf().setMaster("yarn-client").setAppName("app").set("spark.executor.memory", "800M"))

In [2]:
END_STATE = sc.broadcast(100)
END_STATE_NAME = sc.broadcast("End")
START_STATE = sc.broadcast(0)
STATE_NAMES = sc.broadcast(['Start','frontpage','news','tech','local','opinion','on-air','misc','weather','msn-news','health','living','business','msn-sports','sports','summary','bbs','travel'])
num_partitions = 10
num_clusters = sc.broadcast(10)

In [7]:
def pad_traj(traj):
    return [START_STATE.value]+list(traj)+[END_STATE.value,END_STATE.value]
def unpad_traj(traj):
    return traj[1:-2]
def state_to_name(state):
    if state==END_STATE:
        return END_STATE_NAME
    else:
        return STATE_NAMES[state]
def traj_to_namedstate(traj):
    return map(state_to_name,traj)

In [8]:
EA_SMOOTH_TRANSITION = 0.1

def create_smooth_transitions(states=[]):
    return {(i,j):EA_SMOOTH_TRANSITION for i in states for j in states}

In [10]:
def emit_ind_trans_prop(ind__traj__clsp_rob, smooth_trans_mtrx):
    '''
    >>> ind__traj__clsp_rob = (0, ([0, 1, 1, 100, 100], array([ 0.50891031,  0.26625345,  0.22483623]))))
    >>> emit_ind_trans_prop(ind__traj__clsp_rob)
    [((0, (0, 1)), 0.50891030999999998), ((0, (1, 1)), 0.50891030999999998), ((0, (1, 100)), 0.50891030999999998), ((0, (100, 100)), 0.50891030999999998), ((1, (0, 1)), 0.26625345), ((1, (1, 1)), 0.26625345), ((1, (1, 100)), 0.26625345), ((1, (100, 100)), 0.26625345), ((2, (0, 1)), 0.22483623), ((2, (1, 1)), 0.22483623), ((2, (1, 100)), 0.22483623), ((2, (100, 100)), 0.22483623)]
    '''
    index = ind__traj__clsp_rob[0]
    traj = ind__traj__clsp_rob[1][0]
    clusters_probs = ind__traj__clsp_rob[1][1]
    
    cl_step_propbs = []
    
    trans = zip(traj[:-1],traj[1:])
    for c_ind, cls_p in enumerate(clusters_probs):
        for step in trans:        

            ini_p = smooth_trans_mtrx[step]
            cl_step_propbs.append(((c_ind, step), (ini_p + cls_p, index)))
            #             yield ((c_ind, step), cls_p)
        
    return cl_step_propbs

In [11]:
def sort_list_based_on_another(base, dependent):
    '''
    >>> sort_list_based_on_another([0, 2, 1], [0.22, 0.55, 0.66])
    ([0, 1, 2], [0.22, 0.66, 0.55])
    '''
    z = zip(base, dependent)
    sz = sorted(z)
    return [t[0] for t in sz], [t[1] for t in sz]

In [12]:
msnbc_no_header = sc.textFile("/ea/msnbc_no_header.seq", num_partitions)

In [13]:
#read trajectories
list_traj = msnbc_no_header.map(lambda line: [int(i) for i in line.split()]) \
                           .filter(lambda seq: len(seq) <= 500) \
                           .map(pad_traj).zipWithIndex() \
                           .map(lambda t: (t[1], t[0]))

In [14]:
def em_train2(list_traj, num_iterations=5):
    
    num_traj = list_traj.count()
    
    # measure_relative_state_size = states prior probs
    states = list_traj.flatMap(lambda states: states[1])
    states_counts = states.countByValue()
    states_counts_sum = sum(states_counts.values())
    states = {k:float(v)/float(states_counts_sum)  for k,v in states_counts.iteritems()}
    smooth_mtrx = sc.broadcast(create_smooth_transitions(states))
    
    #init trajectories probs (init_traj_probs
    # list_traj_probs = sc.parallelize(np.random.rand(num_traj,3),num_partitions).map(lambda vec: vec/np.sum(vec))
    list_traj_probs = sc.parallelize(xrange(num_traj), num_partitions) \
                                .map(lambda v: np.random.rand(1,num_clusters.value)[0]) \
                                .map(lambda vec: vec/np \
                                .sum(vec)).zipWithIndex() \
                                .map(lambda t: (t[1], t[0]))
    
#     global mikita_markov_models
    for i in range(num_iterations):
        
        print str(i) + ' iteration: ' + strftime("%Y-%m-%d %H:%M:%S", gmtime())
        
        # join trajectories and clusters initial random probs
        list_traj__traj_probs = list_traj.join(list_traj_probs) 
        
        # ((cluster, trans), (p, index))
        # [((0, (0, 1)), (0.24919889991572008, 0)), ((0, (1, 1)), (0.24919889991572008, 0))]
        list_cls_trans__p_index = list_traj__traj_probs.flatMap(lambda t: emit_ind_trans_prop(t, smooth_mtrx.value))
        list_cls_trans__p_index.persist(StorageLevel.MEMORY_AND_DISK)
        
        #################################  CALC MARKOV   ##############################
        
        # ((claster, from), (trans, p))
        # [((0, 0), ((0, 3), 0.45565240865632561)), ((0, 3), ((3, 100), 0.45565240865632561))]
        list_cls_from__trans_p = list_cls_trans__p_index.map(lambda t: ((t[0][0],t[0][1][0]), (t[0][1],t[1][0])))

        # ((claster, from), p)
        # [((0, 0), 0.45565240865632561), ((0, 3), 0.45565240865632561)]
        list_cls_from__p = list_cls_from__trans_p.map(lambda t: (t[0], t[1][1]))
        # ((claster, from), summ)
        # [((1, 3), 88045.381738379481), ((1, 13), 93509.65969107172), ((0, 100), 428820.63873856084), ((2, 10), 53400.141021192758)]
        cls_from__summ = list_cls_from__p.reduceByKey(lambda x,y: x+y)

        # ((cluster,trans), sum)
        # [((2, (16, 100)), 452.38469331913268), ((0, (15, 5)), 1081.6800474165561), ((0, (8, 8)), 142241.31289751496)]
        cls_trans__summ = list_cls_trans__p_index.map(lambda t: ((t[0][0],t[0][1]), t[1][0])) \
                                                .reduceByKey(lambda x,y: x+y)


        #  ((cluster, from), ((trans, trans_sum), from_summ))
        # [((2, 12), (((12, 10), 1019.2083959882245), 113267.37822605985)), ((2, 12), (((12, 2), 3666.330047976217), 113267.37822605985))
        cls_from__transsum_summ = cls_trans__summ.map(lambda t: ((t[0][0],t[0][1][0]), (t[0][1], t[1]))) \
                                                .join(cls_from__summ)

        # [((2, (0, 4)), 0.05168807562712003), ((2, (0, 16)), 0.00042905633895048338), ((2, (0, 6)), 0.16451598030626441)]
        markov_models = cls_from__transsum_summ.map(lambda t: ((t[0][0], t[1][0][0]), t[1][0][1]/t[1][1]))

        
        #################################  CALC NEW PROBS   ##############################
        
        
        # [((2, (16, 100)), ((0.42948178892760447, 264), 0.043625160028027611)), ((2, (16, 100)), ((0.52153298549282001, 262602), 0.043625160028027611))]
        list_cls_trans__p_index_mrkp = list_cls_trans__p_index.join(markov_models)
#         list_cls_trans__p_index_mrkp.persist(StorageLevel.MEMORY_AND_DISK)
        
        # [((0, 789061), ((11, 7), -5.004076355127463)), ((0, 789061), ((11, 7), -5.004076355127463))]
        list_cls_index__trans_logmrkp = list_cls_trans__p_index_mrkp.map(lambda t: ((t[0][0],t[1][0][1]), (t[0][1], math.log(t[1][1]))))

        # walk_logprobs
        # ((cluster, traj_index), [(trans, logP),((trans, logP))])
        # [ ((0, 784470), [((9, 100), -1.3163148315938333), ((0, 9), -2.717692054383914), ((100, 100), 0.0)])]
        list_cls_index__grp_trans_logmrkp = list_cls_index__trans_logmrkp.groupByKey()
        
        # calc_walk_probs
        # (cluster, (traj_index, walk_logprob))
        # [(784470, (0, -4.0340068859777469)), (950019, (1, -2.8974930003028816))
        cls_index__sumlogmrkp = list_cls_index__grp_trans_logmrkp.map(lambda t: (t[0][1], (t[0][0],sum([tr_logp[1] for tr_logp in t[1]]))))
        
        # [(0, [(0, -3.7289041683043873), (2, -3.7284874541593469), (1, -3.7306682864068987)], (655362, <pyspark.resultiterable.ResultIterable object at 0x7f01bf534790>)]
        index__cls_sumlogmrkp = cls_index__sumlogmrkp.groupByKey()
        
        # cluster_probs = logprobs_to_normprobs_safe
        # (index, [p,p,p])
        # [(655362, [0.3332682408091523, 0.3329057626978422, 0.333825996493006]), ]
        list_traj_probs = index__cls_sumlogmrkp.map(lambda t: (t[0], \
                                    ([c_logp[0] for c_logp in t[1]], \
                                     logprobs_to_normprobs_safe(array([c_logp[1] for c_logp in t[1]]))))) \
                                  .map(lambda t: (t[0], (sort_list_based_on_another(t[1][0], t[1][1]))[1] ))
        
        list_cls_trans__p_index.unpersist()
    
    return markov_models

In [15]:
print strftime("%Y-%m-%d %H:%M:%S", gmtime())
mk = em_train2(list_traj, 2)
print mk.collect()

2015-07-21 11:20:32
0 iteration: 2015-07-21 11:20:46
1 iteration: 2015-07-21 11:20:46
[((4, (6, 15)), 0.028092299732725954), ((4, (6, 5)), 0.0064545505466707068), ((4, (6, 17)), 0.00048422813521335843), ((4, (6, 3)), 0.014225500696498652), ((4, (6, 4)), 0.01712375533756668), ((4, (6, 16)), 0.00074821790965420561), ((4, (6, 9)), 0.018689281807386431), ((4, (6, 2)), 0.025790643211069844), ((4, (6, 7)), 0.089699925877285883), ((4, (6, 10)), 0.015182890299053266), ((4, (6, 6)), 0.34185299325140517), ((4, (6, 1)), 0.044382432569236074), ((4, (6, 13)), 0.0068734319990833856), ((4, (6, 12)), 0.0096650989139263795), ((4, (6, 100)), 0.3551998864648479), ((4, (6, 11)), 0.0066287341519118336), ((4, (6, 8)), 0.010728803124366193), ((4, (6, 14)), 0.0081773259721145651), ((6, (3, 100)), 0.39263676050589746), ((6, (3, 11)), 0.014493261588157801), ((6, (3, 12)), 0.020302449695771559), ((6, (3, 7)), 0.0035265198758954739), ((6, (3, 17)), 0.003169157162430498), ((6, (3, 3)), 0.33733642438399475), ((6, (

In [22]:
kk = mk.map(lambda t: (t[0][0], (t[0][1],t[1]))).groupByKey()
markovs = kk.collect()

[(0, <pyspark.resultiterable.ResultIterable object at 0x7f81a4d55f10>), (1, <pyspark.resultiterable.ResultIterable object at 0x7f819ef01710>), (2, <pyspark.resultiterable.ResultIterable object at 0x7f819eee7c50>), (3, <pyspark.resultiterable.ResultIterable object at 0x7f819eee79d0>), (4, <pyspark.resultiterable.ResultIterable object at 0x7f819ff57ed0>), (5, <pyspark.resultiterable.ResultIterable object at 0x7f819ff57750>), (6, <pyspark.resultiterable.ResultIterable object at 0x7f819ff57fd0>), (7, <pyspark.resultiterable.ResultIterable object at 0x7f819ff57690>), (8, <pyspark.resultiterable.ResultIterable object at 0x7f819ff67550>), (9, <pyspark.resultiterable.ResultIterable object at 0x7f819eee7d10>)]

In [ ]:
init = sc.parallelize([1]*100, 3)

for i in range(5):
    print i
    c = sc.broadcast(1)
    init = init.map(lambda n: n+c.value)
    
init.collect()